In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import random
import json
import gc

import PIL
from PIL import Image
import numpy as np
import pandas as pd
import torch
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
from omegaconf import OmegaConf

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
sys.path.insert(1, str(Path(dir1) / 'taming-transformers'))
    
from research.data.natural_scenes import NaturalScenesDataset
from research.experiments.nsd.nsd_access import NSDAccess
from research.metrics.metrics import cosine_distance, top_knn_test

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker
from transformers import AutoFeatureExtractor

In [ ]:
# load safety model
safety_model_id = "CompVis/stable-diffusion-safety-checker"
safety_feature_extractor = AutoFeatureExtractor.from_pretrained(safety_model_id)
safety_checker = StableDiffusionSafetyChecker.from_pretrained(safety_model_id)

In [ ]:
import ldm

stable_diffusion_path = Path(ldm.__file__).parent.parent

ckpt = 'X:\\Models\\stable-diffusion\\sd-v1-4.ckpt'
config = OmegaConf.load(stable_diffusion_path / 'configs/stable-diffusion/v1-inference.yaml')

pl_sd = torch.load(ckpt, map_location="cpu")
if "global_step" in pl_sd:
    print(f"Global Step: {pl_sd['global_step']}")
    
sd = pl_sd["state_dict"]
model = instantiate_from_config(config.model)
m, u = model.load_state_dict(sd, strict=False)

if len(m) > 0 and verbose:
    print("missing keys:")
    print(m)
if len(u) > 0 and verbose:
    print("unexpected keys:")
    print(u)

model.cuda()
model.eval()

sampler = DDIMSampler(model)

In [ ]:
from taming.modules.vqvae.quantize import VectorQuantizer2 as VectorQuantizer